# Calculation of RSE and quantitative determination of PFAS concentrations from the LC-TQ
Author: Valerie de Rijk

Date: 15/8/2025

## 1 Data Loading
First we load in the raw data as a csv file (which is the file you directly get from MassHunter).

In [1]:
%run src/functions_quantitative.ipynb
rawdata = load_and_label_pfas_csv("rawdata/20250828_AllResults_v4.csv")

Detected delimiter: ','
['PFBA', '13C4-PFBA', 'PFPeA', '13C5-PFPeA', 'HFPO-DA', '13C3-HFPO-DA', 'PFBS', '13C3-PFBS', 'PFHxA', '13C5-PFHxA', '4:2 FTS', '13C2-4:2FTS', 'PFPeS', 'PFPHpA', '13C4-PFHpA', 'ADONA', 'PFHxS', '13C3-PFHxS', 'PFOA', '13C8-PFOA', '6:2 FTS', '13C2-6:2FTS', 'PFHpS', 'PFNA', '13C9-PFNA', 'PFOSA', 'PFOS', '13C8-PFOSA', '13C8-PFOS', 'NMeFOSA', 'PFDA', 'D3-NMeFOSA', '13C6-PFDA', 'NEtFOSA', '8:2 FTS', '13C2-8:2FTS', '9Cl-PF3ONS', 'D5-NEtFOSA', 'PFNS', 'PFUnA', 'NMeFOSAA', '13C7-PFUnA', 'D3-NMeFOSAA', 'NEtFOSAA', 'D5-NEtFOSAA', 'PFDS', 'PFDoA', '13C2-PFDoA', 'NMeFOSE', 'D7-NMeFOSE', 'NEtFOSE', '11Cl-PF3OUdS', 'D9-NEtFOSE', 'PFTrDA', 'PFTeDA', '13C2-PFTeDA']
Total compounds: 56
Without isotope prefix: 32


## 2  Calibration Line (Instrument linearity)
To analyze all instrument linearity metrics, we first need to calculate the concentrations found in the samples. 
We do this by caclulating the concentrations based on the formula on page 44 of EPA method 1633. 
To achieve this, we first need to calculate the average response factor per compound. 


We compute the RSE based on  the formula for RSE based on EPA method 1633 (section 10.3.3.3, page 28, option 2). The functions are loaded in a different script. 
If the RSE remains above 20%, this means something is wrong with the linearity of the LC-MS TQ and the calculated concentrations in section 3 are not trustworthy. 

*We mute the output of this cell for clarity of the script and summarize them under EIS results and Target Analytes results. If interested in full computation, please unmute the cell by removing the cell line **%%capture***

The output is structured as following: 
- List of identified EIS compounds (check if all are found)
- Computation for the EIS compounds
- Computation for the target analytes
- Summary of the result for EIS
- Summary of the result for target

*note: if not interested in detailed calculations, run the cell and ignore the output*


In [2]:
%%capture
%run src/functions_quantitative.ipynb
import importlib
import expected_concs
from expected_concs import expected_concs_EIS, expected_concs_target_analytes

from analogs_compounds import target_EIS_analogs, EIS_NIS_analogs
from solutions import calibration_solutions

# computation of EIS concentration and recovery rate
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous

eis_RFS = calculate_average_RFs_EIS(rawdata, EIS_NIS_analogs, calibration_solutions)


Ws_cal = 0.25#ml
dilution_NIS_stock_cal = 40 #dilution cal
dilution_EIS_stock_cal = 40 #dilution cal
spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount = 0.05 #ml

dilution_EIS_stock = 2 
spiked_amount_EIS =0.05 #ml

#calibration line
conc_EIS_cal = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)

calculated_recoveries_EIS_cal = calculate_recoveries_EIS(conc_EIS_cal, EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal)

rse_results_cal_EIS = calculate_eis_rse_modified(conc_EIS_cal, expected_concs_EIS, p=2)
validation_results_cal= validate_recoveries(calculated_recoveries_EIS_cal, EIS_NIS_compound_recovery_aqueous)
summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_cal = get_failed_recoveries(validation_results_cal)



# Example run
#rse_results_eis = calculate_eis_rse(rawdata, expected_concs_EIS)
#rse_results_target = calculate_target_rse(rawdata, expected_concs_target_analytes)

#print_rse_summary(rse_results_eis)
#print_rse_summary(rse_results_target)

NameError: name 'calculate_recoveries_EIS' is not defined

### 2.1 EIS results

We print the results in a clear table and save them to csv (first for EIS, then target). Give the files an intuitive name.

In [3]:
df_rse_eis=df_rse_results(rse_results_cal_EIS)
display(df_rse_eis)

,Compound,RSE_Percent,Calibration_Points,Relationship_Type,Parameters,Pass_Fail,Sum_Relative_Squared_Error
0,13C2-PFDoA,8.323322,7,linear,2,PASS,0.006928
1,13C8-PFOA,3.154659,7,linear,2,PASS,0.000995
2,13C2-6:2FTS,10.675406,7,linear,2,PASS,0.011396
3,13C9-PFNA,2.895678,7,linear,2,PASS,0.000838
4,13C5-PFHxA,0.909806,7,linear,2,PASS,0.000083
5,13C4-PFBA,0.383990,7,linear,2,PASS,0.000015
6,13C3-PFHxS,2.561264,7,linear,2,PASS,0.000656
7,13C3-HFPO-DA,1.599224,7,linear,2,PASS,0.000256
8,13C3-PFBS,2.989945,7,linear,2,PASS,0.000894
9,13C2-PFTeDA,7.067921,7,linear,2,PASS,0.004996


### 2.2 Target Analytes results

In [6]:
df_rse_target = df_rse_results(rse_results_target)
display(df_rse_target)

,Compound,RSE_Percent,Calibration_Points,Relationship_Type,Parameters,Pass_Fail,Sum_Relative_Squared_Error
0,PFBA,13.126824,7,linear,2,PASS,0.017231
1,PFPeA,12.245095,7,linear,2,PASS,0.014994
2,HFPO-DA,11.736307,7,linear,2,PASS,0.013774
3,PFBS,12.797400,7,linear,2,PASS,0.016377
4,PFHxA,10.595762,7,linear,2,PASS,0.011227
5,4:2 FTS,27.750370,6,quadratic,3,FAIL,0.077008
6,PFPeS,15.716212,7,linear,2,PASS,0.024700
7,ADONA,14.804778,7,linear,2,PASS,0.021918
8,PFHxS,16.833835,7,linear,2,PASS,0.028338
9,PFOA,14.110585,7,linear,2,PASS,0.019911


## 3 Instrument checks
*all of these calculations still need to be added*

We perform different checks apart from instrument linearity to ensure that the LC-TQ is behaving as we expect. 
1. Instrument Sensitivity check (signal to noise ratio)
     - here we check if the ratio is greater than or equal to 3:1 (EPA section 14.1)
2. Bile salt interference check
   - We check that the RT of bile salts falls at least one minute outside of the retention time for any linear or branched PFOS (EPA 14.2)
3. Calibration verification (EPA 14.3)
   - Check that the CV, e.g. the mid-level calibration point, (which should be done every 10 samples or less) has a recovery between 70 and 130%.
4. Retention times
    - All target analyte with  true isotope dilution should elute within 0.1 minutes of the associated EIS compounds.
5. Ongoing precision and recovery
    - The recovery of native compounds and EIS compounds must meet the standards in table 5,6,7 and 8 of the EPA.
6. Instrument blank and method blank
   - Check that is it below all analyte's MDL.
7. Instrument sensitivity
   - This is the ratio of the NIS peak areas from QC and field samples relative to the mean area of the corresponding NIS in most recent inital calibration. This must be within 50 to 200%. 


## 3.1 recovery  EIS compounds
This follows EPA method 1633 page 44 and EPA method page 28 (equation for RFs). We first compute the average response factor for the EIS compounds in the calibration standard. Then, we use the formula on page 44 to calculate the concentration in the sample and compare it to the expected concentration to determine the recovery. 

We also check the used concentration from above. 

### 3.1.1 Calculating average Response Factor EIS compounds

In [2]:
from analogs_compounds import target_EIS_analogs, EIS_NIS_analogs
from solutions import calibration_solutions

eis_RFS = calculate_average_RFs_EIS(rawdata, EIS_NIS_analogs, calibration_solutions)


AVERAGE RESPONSE FACTORS FOR EIS COMPOUNDS (n=24)


,Compound_Name,NIS_Analog,Mass_NIS,Mass_EIS,Average_Response_Factor,RSD_%
0,13C3-PFHxS,18O2-PFHxS,2.50,2.50,1.8312,173.43
1,13C5-PFHxA,13C2-PFHxA,2.50,2.50,2.0985,173.16
2,13C2-PFDoA,13C2-PFDA,1.25,1.25,3.8255,171.54
3,13C5-PFPeA,13C2-PFHxA,2.50,5.00,1.0163,173.06
4,13C2-6:2FTS,18O2-PFHxS,2.50,5.00,4.8377,163.79
5,D5-NEtFOSAA,13C4-PFOS,2.50,5.00,0.4679,170.42
6,D5-NEtFOSA,13C4-PFOS,2.50,2.50,8.8892,170.34
7,13C4-PFHpA,13C2-PFHxA,2.50,2.50,1.9642,172.48
8,13C9-PFNA,13C5-PFNA,1.25,1.25,4.0900,173.79
9,13C3-HFPO-DA,13C2-PFHxA,2.50,10.00,0.5044,171.98


### 3.1.2 Concentration and recovery of EIS compounds 
*TO DO the weights of the bottles should be added as a column per sample*
*TO DO make a check if recovery falls in Table 5 + add RSD*


Here we compute the concentration of the EIS compounds based on the calculated response factor, the dilution factor and the sample volume (EPA 1633, page 44). 
It is important to enter the correct sample volume, otherwise calculations will be wrong. 

In [24]:
# computation of EIS concentration and recovery rate
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous
DF_sample = 1 #dilution factor 
mass_bottle_filled = 499.8 #grams
mass_bottle_empty = 53.68 #grams
Ws = (mass_bottle_filled - mass_bottle_empty)/1000 #L
Ws_cal = 0.25#ml


dilution_NIS_stock = 2 #1:1 dilution
dilution_NIS_stock_cal = 40 #dilution cal
dilution_EIS_stock_cal = 40 #dilution cal
spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount = 0.05 #ml

dilution_EIS_stock = 2 
spiked_amount_EIS =0.05 #ml

#calibration line
conc_EIS_cal = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)

calculated_recoveries_EIS_cal = calculate_recoveries_EIS(conc_EIS_cal, EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal)

rse_results_Cal_EIS = calculate_eis_rse_modified(conc_EIS_cal, expected_concs_EIS, p=2)
validation_results_cal= validate_recoveries(calculated_recoveries_EIS_cal, EIS_NIS_compound_recovery_aqueous)
summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_cal = get_failed_recoveries(validation_results_cal)



calculated_cons_EIS = calculate_conc_EIS(rawdata, eis_RFS, dilution_NIS_stock, Ws,EIS_NIS_analogs, DF_sample, NIS_stock, spiked_amount)

calculated_recoveries_EIS = calculate_recoveries_EIS(calculated_cons_EIS, EIS_stock, dilution_EIS_stock, spiked_amount_EIS, Ws)


conc_EIS_cal = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)
# Run the validation
validation_results = validate_recoveries(calculated_recoveries_EIS, EIS_NIS_compound_recovery_aqueous)

# Get summary
summary = summarize_validation_results(validation_results)
print("Validation Summary:")
for key, value in summary['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries = get_failed_recoveries(validation_results)

,Name,Type,13C6-PFDA_Conc_Calc,13C2-PFTeDA_Conc_Calc,13C3-PFBS_Conc_Calc,13C7-PFUnA_Conc_Calc,13C9-PFNA_Conc_Calc,13C3-PFHxS_Conc_Calc,13C2-8:2FTS_Conc_Calc,13C8-PFOS_Conc_Calc,13C2-6:2FTS_Conc_Calc,D9-NEtFOSE_Conc_Calc,D3-NMeFOSA_Conc_Calc,D5-NEtFOSA_Conc_Calc,13C2-4:2FTS_Conc_Calc,13C4-PFHpA_Conc_Calc,D3-NMeFOSAA_Conc_Calc,13C5-PFHxA_Conc_Calc,13C8-PFOSA_Conc_Calc,13C5-PFPeA_Conc_Calc,13C2-PFDoA_Conc_Calc,13C3-HFPO-DA_Conc_Calc,13C8-PFOA_Conc_Calc,D5-NEtFOSAA_Conc_Calc,D7-NMeFOSE_Conc_Calc,13C4-PFBA_Conc_Calc
1,CS1,Cal,1.139074,1.126445,2.556475,1.194792,1.183689,2.588120,4.246637,2.663938,4.341826,25.340473,2.378188,2.378428,4.177484,2.444687,5.125671,2.484035,2.480380,4.912019,1.173739,9.905255,2.539149,4.984211,24.989409,10.012923
2,CS2,Cal,1.232340,1.192970,2.539901,1.215018,1.280387,2.446245,4.035874,2.600694,4.124204,31.612544,3.174787,3.058330,4.111333,2.409584,4.759639,2.513343,2.660696,5.048464,1.153753,10.125552,2.404669,4.627632,31.330097,10.049465
3,CS3,Cal,1.355756,1.205650,2.479252,1.437871,1.260816,2.522462,5.894619,2.479167,5.229565,22.439352,2.216502,2.175399,4.309785,2.581108,4.984139,2.493567,2.310127,4.764017,1.390928,9.981981,2.407309,5.032895,21.875110,9.949198
4,CS4,Cal,1.323391,1.350511,2.551687,1.337943,1.241759,2.432806,4.289238,2.466270,4.523620,24.328267,2.337438,2.380270,4.398402,2.508510,4.864568,2.523351,2.469810,4.995899,1.370278,10.116252,2.560582,5.060526,24.233892,10.007130
5,CS5,Cal,1.205415,1.258497,2.498404,1.215467,1.265709,2.458340,4.210762,2.408482,4.166667,24.670275,2.358251,2.401965,4.458937,2.389905,4.697413,2.505719,2.483608,4.980987,1.202360,10.106371,2.547186,5.344737,24.176081,9.958111
6,CS6,Cal,1.232408,1.279255,2.514855,1.197264,1.276782,2.555483,4.786996,2.635169,4.869958,25.957316,2.630357,2.654318,5.140414,2.454792,5.494143,2.515488,2.677025,5.074187,1.257789,10.041851,2.571259,5.147368,26.442630,10.019162
7,CS7,Cal,1.261415,1.336672,2.359181,1.151495,1.240987,2.496544,7.526906,2.246528,7.737527,20.641409,2.404916,2.450880,8.399900,2.711414,5.068326,2.464020,2.418354,5.223307,1.201152,9.723901,2.469787,4.794737,21.941748,10.002228


,Name,Type,D7-NMeFOSE_Recovery_%,D9-NEtFOSE_Recovery_%,13C4-PFBA_Recovery_%,13C3-HFPO-DA_Recovery_%,13C5-PFPeA_Recovery_%,13C2-4:2FTS_Recovery_%,13C2-6:2FTS_Recovery_%,13C2-8:2FTS_Recovery_%,D3-NMeFOSAA_Recovery_%,D5-NEtFOSAA_Recovery_%,13C5-PFHxA_Recovery_%,13C4-PFHpA_Recovery_%,13C8-PFOA_Recovery_%,13C3-PFBS_Recovery_%,13C3-PFHxS_Recovery_%,13C8-PFOS_Recovery_%,13C8-PFOSA_Recovery_%,D3-NMeFOSA_Recovery_%,D5-NEtFOSA_Recovery_%,13C9-PFNA_Recovery_%,13C6-PFDA_Recovery_%,13C7-PFUnA_Recovery_%,13C2-PFDoA_Recovery_%,13C2-PFTeDA_Recovery_%
1,CS1,Cal,99.957635,101.361892,100.129234,99.052546,98.240382,83.549675,86.836518,84.932735,102.513421,99.684211,99.361418,97.787469,101.565971,102.258999,103.524804,106.557540,99.215190,95.127509,95.137126,94.695097,91.125913,95.583388,93.899140,90.115605
2,CS2,Cal,125.320388,126.450177,100.494652,101.255522,100.969281,82.226660,82.484076,80.717489,95.192777,92.552632,100.533740,96.383363,96.186759,101.596032,97.849793,104.027778,106.427848,126.991499,122.333197,102.430985,98.587237,97.201414,92.300261,95.437576
3,CS3,Cal,87.500441,89.757408,99.491979,99.819809,95.280346,86.195706,104.591295,117.892377,99.682772,100.657895,99.742661,103.244336,96.292356,99.170063,100.898479,99.166667,92.405063,88.660065,87.015964,100.865266,108.460464,115.029664,111.274273,96.452020
4,CS4,Cal,96.935569,97.313066,100.071301,101.162520,99.917984,87.968048,90.472399,85.784753,97.291362,101.210526,100.934045,100.340389,102.423266,102.067475,97.312241,98.650794,98.792405,93.497502,95.210806,99.340750,105.871294,107.035417,109.622259,108.040887
5,CS5,Cal,96.704325,98.681099,99.581105,101.063706,99.619744,89.178732,83.333333,84.215247,93.948267,106.894737,100.228746,95.596213,101.887457,99.936159,98.333589,96.339286,99.344304,94.330032,96.078592,101.256696,96.433176,97.237370,96.188774,100.679729
6,CS6,Cal,105.770521,103.829265,100.191622,100.418507,101.483746,102.808288,97.399151,95.739910,109.882870,102.947368,100.619520,98.191682,102.850349,100.594215,102.219321,105.406746,107.081013,105.214267,106.172738,102.142563,98.592609,95.781147,100.623128,102.340431
7,CS7,Cal,87.766990,82.565638,100.022282,97.239014,104.466150,167.998003,154.750531,150.538117,101.366520,95.894737,98.560808,108.456547,98.791496,94.367235,99.861772,89.861111,96.734177,96.196652,98.035203,99.278945,100.913193,92.119614,96.092165,106.933752


EIS compounds found: 24
EIS compounds with expected concentrations: 24
Number of Cal samples found: 7

13C6-PFDA (p=2)
Point	Expected	Measured	Difference	(Diff)²/xi²	(Diff)²/xi²/(n-p)
--------------------------------------------------------------------------------
CAL 1	1.25		1.1391		-0.1109		0.007875	0.001575
CAL 2	1.25		1.2323		-0.0177		0.000200	0.000040
CAL 3	1.25		1.3558		0.1058		0.007158	0.001432
CAL 4	1.25		1.3234		0.0734		0.003447	0.000689
CAL 5	1.25		1.2054		-0.0446		0.001272	0.000254
CAL 6	1.25		1.2324		-0.0176		0.000198	0.000040
CAL 7	1.25		1.2614		0.0114		0.000083	0.000017
--------------------------------------------------------------------------------
Sum of relative squared errors: 0.004047
RSE = 100 * √(0.004047) = 6.361%

13C2-PFTeDA (p=2)
Point	Expected	Measured	Difference	(Diff)²/xi²	(Diff)²/xi²/(n-p)
--------------------------------------------------------------------------------
CAL 1	1.25		1.1264		-0.1236		0.009770	0.001954
CAL 2	1.25		1.1930		-0.0570		0.002082	0.00

,Sample_Name,Sample_Type,Compound,Calculated_Recovery_%,Expected_Lower_%,Expected_Upper_%,Status,Compound_Type


,Name,Type,13C6-PFDA_Conc_Calc,13C2-PFTeDA_Conc_Calc,13C3-PFBS_Conc_Calc,13C7-PFUnA_Conc_Calc,13C9-PFNA_Conc_Calc,13C3-PFHxS_Conc_Calc,13C2-8:2FTS_Conc_Calc,13C8-PFOS_Conc_Calc,13C2-6:2FTS_Conc_Calc,D9-NEtFOSE_Conc_Calc,D3-NMeFOSA_Conc_Calc,D5-NEtFOSA_Conc_Calc,13C2-4:2FTS_Conc_Calc,13C4-PFHpA_Conc_Calc,D3-NMeFOSAA_Conc_Calc,13C5-PFHxA_Conc_Calc,13C8-PFOSA_Conc_Calc,13C5-PFPeA_Conc_Calc,13C2-PFDoA_Conc_Calc,13C3-HFPO-DA_Conc_Calc,13C8-PFOA_Conc_Calc,D5-NEtFOSAA_Conc_Calc,D7-NMeFOSE_Conc_Calc,13C4-PFBA_Conc_Calc


,Name,Type,D7-NMeFOSE_Recovery_%,D9-NEtFOSE_Recovery_%,13C4-PFBA_Recovery_%,13C3-HFPO-DA_Recovery_%,13C5-PFPeA_Recovery_%,13C2-4:2FTS_Recovery_%,13C2-6:2FTS_Recovery_%,13C2-8:2FTS_Recovery_%,D3-NMeFOSAA_Recovery_%,D5-NEtFOSAA_Recovery_%,13C5-PFHxA_Recovery_%,13C4-PFHpA_Recovery_%,13C8-PFOA_Recovery_%,13C3-PFBS_Recovery_%,13C3-PFHxS_Recovery_%,13C8-PFOS_Recovery_%,13C8-PFOSA_Recovery_%,D3-NMeFOSA_Recovery_%,D5-NEtFOSA_Recovery_%,13C9-PFNA_Recovery_%,13C6-PFDA_Recovery_%,13C7-PFUnA_Recovery_%,13C2-PFDoA_Recovery_%,13C2-PFTeDA_Recovery_%


,Name,Type,13C6-PFDA_Conc_Calc,13C2-PFTeDA_Conc_Calc,13C3-PFBS_Conc_Calc,13C7-PFUnA_Conc_Calc,13C9-PFNA_Conc_Calc,13C3-PFHxS_Conc_Calc,13C2-8:2FTS_Conc_Calc,13C8-PFOS_Conc_Calc,13C2-6:2FTS_Conc_Calc,D9-NEtFOSE_Conc_Calc,D3-NMeFOSA_Conc_Calc,D5-NEtFOSA_Conc_Calc,13C2-4:2FTS_Conc_Calc,13C4-PFHpA_Conc_Calc,D3-NMeFOSAA_Conc_Calc,13C5-PFHxA_Conc_Calc,13C8-PFOSA_Conc_Calc,13C5-PFPeA_Conc_Calc,13C2-PFDoA_Conc_Calc,13C3-HFPO-DA_Conc_Calc,13C8-PFOA_Conc_Calc,D5-NEtFOSAA_Conc_Calc,D7-NMeFOSE_Conc_Calc,13C4-PFBA_Conc_Calc
1,CS1,Cal,1.139074,1.126445,2.556475,1.194792,1.183689,2.588120,4.246637,2.663938,4.341826,25.340473,2.378188,2.378428,4.177484,2.444687,5.125671,2.484035,2.480380,4.912019,1.173739,9.905255,2.539149,4.984211,24.989409,10.012923
2,CS2,Cal,1.232340,1.192970,2.539901,1.215018,1.280387,2.446245,4.035874,2.600694,4.124204,31.612544,3.174787,3.058330,4.111333,2.409584,4.759639,2.513343,2.660696,5.048464,1.153753,10.125552,2.404669,4.627632,31.330097,10.049465
3,CS3,Cal,1.355756,1.205650,2.479252,1.437871,1.260816,2.522462,5.894619,2.479167,5.229565,22.439352,2.216502,2.175399,4.309785,2.581108,4.984139,2.493567,2.310127,4.764017,1.390928,9.981981,2.407309,5.032895,21.875110,9.949198
4,CS4,Cal,1.323391,1.350511,2.551687,1.337943,1.241759,2.432806,4.289238,2.466270,4.523620,24.328267,2.337438,2.380270,4.398402,2.508510,4.864568,2.523351,2.469810,4.995899,1.370278,10.116252,2.560582,5.060526,24.233892,10.007130
5,CS5,Cal,1.205415,1.258497,2.498404,1.215467,1.265709,2.458340,4.210762,2.408482,4.166667,24.670275,2.358251,2.401965,4.458937,2.389905,4.697413,2.505719,2.483608,4.980987,1.202360,10.106371,2.547186,5.344737,24.176081,9.958111
6,CS6,Cal,1.232408,1.279255,2.514855,1.197264,1.276782,2.555483,4.786996,2.635169,4.869958,25.957316,2.630357,2.654318,5.140414,2.454792,5.494143,2.515488,2.677025,5.074187,1.257789,10.041851,2.571259,5.147368,26.442630,10.019162
7,CS7,Cal,1.261415,1.336672,2.359181,1.151495,1.240987,2.496544,7.526906,2.246528,7.737527,20.641409,2.404916,2.450880,8.399900,2.711414,5.068326,2.464020,2.418354,5.223307,1.201152,9.723901,2.469787,4.794737,21.941748,10.002228


KeyError: 'Status'

## 4 Quantitative determination of concentrations target analytes
In this section we will compute the concentration of all the analyzed PFAS compounds in the sample. Our current method is capable of analyzing 40 different compounds. Here, we distinguish between two categories to perform the calculation: 
- True Isotope Dilution Quantification (ID): 24 compounds are directly compared to their istopically labeled analog.
- Extracted Internal Standard Quantification (EIS): 16 compounds are quantified by comparing to a *different* isotopically labeled compound.

### 4.1 Response Ratio (RR) and Response Factor (RF) for target ID compounds and target EIS compounds.
Here we compute the response ratio for the ID compounds by using the formula from EPA 1633 on page 27 under section 10.3.3.2. For now we also fix some naming errors in the raw data for the target analytes. The only difference with the above calculation is that EIS analogs instead of direct pairs are used for the target EIS compounds. For ease, we call both RF.

- *Note 1*: NMeFoSA and NEtFoSa are calculated now but are not included in the PFAC30PAR ampoule we currently use in the lab, so these compounds cannot be in the calibration line. Use these results with care.
- *Note 2*:EPA 1633 mentions that PFTrDA recovery should improve by taking the average of the analog EIS compounds 13c2-PFTeDA and 13c2-PFDoa. Currently it only takes the EIS compound 13C2-PFTeDA as an analog.
- *Note 3*: EPA 1633 is capable of analyzing 40 compounds, our native standard solution only contains 30, so you will get errors that data for some compounds is not found. This is correct. If new ampoules are ordered for the other 10 compounds, everything needed for the calculation is there.
- *Note 4*: the FTS compounds only have 6 cal levels instead of 7.

In [7]:
from analogs_compounds import target_EIS_analogs

# Make a copy of rawdata to avoid modifying the original
rawdata_copy = rawdata.copy()
columns_to_rename = {}
for col in rawdata_copy.columns:
    new_col = col
    # Fix PFPHpA -> PFHpA
    if 'PFPHpA' in col:
        new_col = new_col.replace('PFPHpA', 'PFHpA')
    # Remove spaces from FTS compounds (e.g., "4:2 FTS" -> "4:2FTS")
    if ' FTS' in col:
        new_col = new_col.replace(' FTS', 'FTS')
    
    if new_col != col:
        columns_to_rename[col] = new_col

if columns_to_rename:
    rawdata_copy.rename(columns=columns_to_rename, inplace=True)

RF_target_analytes= calculate_average_RRs_targets(rawdata_copy, expected_concs_EIS, calibration_solutions, target_EIS_analogs)

Found 40 total target-IS pairs:
  NMeFOSE -> D7-NMeFOSE
  NEtFOSE -> D9-NEtFOSE
  PFBA -> 13C4-PFBA
  HFPO-DA -> 13C3-HFPO-DA
  PFPeA -> 13C5-PFPeA
  4:2FTS -> 13C2-4:2FTS
  6:2FTS -> 13C2-6:2FTS
  8:2FTS -> 13C2-8:2FTS
  NMeFOSAA -> D3-NMeFOSAA
  NEtFOSAA -> D5-NEtFOSAA
  PFHxA -> 13C5-PFHxA
  PFHpA -> 13C4-PFHpA
  PFOA -> 13C8-PFOA
  PFBS -> 13C3-PFBS
  PFHxS -> 13C3-PFHxS
  PFOS -> 13C8-PFOS
  PFOSA -> 13C8-PFOSA
  NMeFOSA -> D3-NMeFOSA
  NEtFOSA -> D5-NEtFOSA
  PFNA -> 13C9-PFNA
  PFDA -> 13C6-PFDA
  PFUnA -> 13C7-PFUnA
  PFDoA -> 13C2-PFDoA
  PFTeDA -> 13C2-PFTeDA
  PFTrDA -> 13C2-PFTeDA
  PFPeS -> 13C3-PFHxS
  PFHpS -> 13C8-PFOS
  PFNS -> 13C8-PFOS
  PFDS -> 13C8-PFOS
  PFDoS -> 13C8-PFOS
  ADONA -> 13C3-HFPO-DA
  PFMPA -> 13C5-PFPeA
  PFMBA -> 13C5-PFPeA
  NFDHA -> 13C5-PFHxA
  9Cl-PF3ONS -> 13C3-HFPO-DA
  11Cl-PF3OUdS -> 13C3-HFPO-DA
  PFEESA -> 13C5-PFHxA
  3:3FTCA -> 13C5-PFPeA
  5:3FTCA -> 13C5-PFHxA
  7:3FTCA -> 13C5-PFHxA


AVERAGE RESPONSE RATIOS FOR TARGET COMPOUNDS WITH

,Target_Compound,EIS_Isotope,Mass_Target (L4),Mass_EIS,Average_Response_Ratio,RSD_%,N_Levels,Levels_Used
0,NMeFOSE,D7-NMeFOSE,25.0,25.00,0.6122,6.13,7,"L1, L2, L3, L4, L5, L6, L7"
1,NEtFOSE,D9-NEtFOSE,25.0,25.00,0.5958,6.58,7,"L1, L2, L3, L4, L5, L6, L7"
2,PFBA,13C4-PFBA,10.0,10.00,0.8575,6.64,7,"L1, L2, L3, L4, L5, L6, L7"
3,HFPO-DA,13C3-HFPO-DA,10.0,10.00,0.8923,9.42,7,"L1, L2, L3, L4, L5, L6, L7"
4,PFPeA,13C5-PFPeA,5.0,5.00,0.9047,6.19,7,"L1, L2, L3, L4, L5, L6, L7"
5,4:2FTS,13C2-4:2FTS,10.0,5.00,0.8746,13.74,6,"L1, L2, L3, L4, L5, L6"
6,6:2FTS,13C2-6:2FTS,10.0,5.00,0.8135,9.46,6,"L1, L2, L3, L4, L5, L6"
7,8:2FTS,13C2-8:2FTS,10.0,5.00,1.0489,15.88,6,"L1, L2, L3, L4, L5, L6"
8,NMeFOSAA,D3-NMeFOSAA,2.5,5.00,0.7380,12.34,7,"L1, L2, L3, L4, L5, L6, L7"
9,NEtFOSAA,D5-NEtFOSAA,2.5,5.00,0.7115,34.11,7,"L1, L2, L3, L4, L5, L6, L7"


### 4.2 Calculating Concentrations target analytes
Here we compute the concentration of the target analytes (ng/L) based on the formula on page 44 of EPA method 1633. 

*Note 1* % solids is not added yet, needs to be done before the solids are added. 


In [31]:

conc_target_analytes = calculate_conc_targets(rawdata_copy,
                           RF_target_analytes,
                           EIS_stock, dilution_EIS_stock, spiked_amount_EIS, Ws,
                           DF_sample,  debug_compound=None )

,Name,Type,NMeFOSE_Conc_Calc,NEtFOSE_Conc_Calc,PFBA_Conc_Calc,HFPO-DA_Conc_Calc,PFPeA_Conc_Calc,4:2FTS_Conc_Calc,6:2FTS_Conc_Calc,8:2FTS_Conc_Calc,NMeFOSAA_Conc_Calc,NEtFOSAA_Conc_Calc,PFHxA_Conc_Calc,PFHpA_Conc_Calc,PFOA_Conc_Calc,PFBS_Conc_Calc,PFHxS_Conc_Calc,PFOS_Conc_Calc,PFOSA_Conc_Calc,NMeFOSA_Conc_Calc,NEtFOSA_Conc_Calc,PFNA_Conc_Calc,PFDA_Conc_Calc,PFUnA_Conc_Calc,PFDoA_Conc_Calc,PFTeDA_Conc_Calc,PFTrDA_Conc_Calc,PFPeS_Conc_Calc,PFHpS_Conc_Calc,PFNS_Conc_Calc,PFDS_Conc_Calc,ADONA_Conc_Calc,9Cl-PF3ONS_Conc_Calc,11Cl-PF3OUdS_Conc_Calc
0,blank + EIS,Sample,0.000000,0.044145,0.048491,0.026670,0.094423,0.000000,0.147824,0.185508,0.105610,0.118083,5.874425e-02,0.054764,0.119739,0.020110,0.191261,0.191355,0.052238,0.019421,0.000000,0.012262,0.019013,0.046180,0.096305,0.029658,0.007029,0.012115,0.000000,0.032334,0.000000,0.010448,0.000000,0.012141
1,QC std,Sample,286.373524,283.263461,112.733975,113.943425,55.842305,118.231661,122.525543,130.884930,34.756022,28.920200,2.918120e+01,28.913236,24.038937,29.522970,27.598264,23.652089,19.312385,15.140355,17.380540,26.747680,27.658844,26.902369,27.545906,27.704749,27.794351,27.732618,27.915531,28.062973,27.392650,114.552926,111.748518,113.542192
2,KCS,Sample,0.186790,0.128562,100.110275,99.440915,98.891708,101.966523,103.684032,112.195006,96.188390,126.538972,9.929793e+01,98.476915,129.482542,94.936553,98.804210,82.953016,101.182899,0.000000,0.000000,124.189381,97.187245,99.159062,102.523510,99.211478,95.371284,94.262425,105.847550,104.084862,100.268761,92.635127,97.586505,94.680232
3,DWZ,Sample,0.081178,0.036357,0.098633,0.031282,0.069084,0.039237,0.315508,0.258098,0.087280,0.366332,1.918389e-01,0.077114,0.486346,0.375910,0.072870,0.427542,0.055238,0.000000,0.000000,0.060937,0.005963,0.030205,0.053893,0.032421,0.016466,0.027695,0.000000,0.000000,0.000000,0.002451,0.001545,0.008544
4,KWZ,Sample,0.048325,0.043027,0.533444,0.033271,0.162249,0.041633,0.210660,0.157136,0.000000,0.216377,6.775727e-01,0.171176,2.184125,0.209707,0.274737,0.168459,2.749355,0.000000,0.000000,0.039106,0.128910,0.023016,0.141501,0.042568,0.036032,0.010237,0.055896,0.000000,0.030244,0.005213,0.004931,0.009088
5,DWP,Sample,0.000000,0.056859,128.225749,128.300387,126.694831,132.234659,124.147537,131.390149,137.510359,176.314414,1.307989e+02,133.522204,164.139903,125.625723,126.995839,101.329103,134.362334,0.013019,0.051849,173.551200,123.763634,127.599454,132.572311,132.030676,136.489525,124.280221,129.053438,130.851465,115.935328,117.879063,115.605196,95.840190
6,KWP,Sample,0.058311,0.106797,114.441712,117.976646,111.821477,117.989636,124.863149,95.452967,104.300351,127.903123,1.169410e+02,117.005643,148.563044,112.846012,107.364570,92.608445,118.051421,0.012351,0.000000,152.655890,112.683144,109.327063,109.746333,114.756823,121.013875,104.335991,121.945902,118.222874,108.424107,108.794084,95.516111,78.988097
7,KCS,Sample,0.046899,0.042714,99.416376,98.966156,98.073562,109.644592,101.196458,103.533989,96.603960,108.829504,1.024862e+02,98.061048,124.214134,97.778281,94.338188,79.555349,103.531608,0.009269,0.000000,126.713645,93.378788,97.074606,98.846420,99.160559,99.972664,96.120326,104.075731,100.463952,96.799751,92.706086,92.789094,90.604086
8,QCstd,Sample,282.964584,282.867399,112.018962,117.177520,56.609012,121.284152,123.500737,136.184414,28.130463,27.727836,2.883969e+01,27.051016,25.598477,29.580441,26.839201,22.758334,19.515384,15.664513,17.595904,28.452699,28.582887,28.022766,28.458948,27.045089,28.381938,27.738460,29.507468,28.317768,28.137940,111.681636,109.461237,108.304356
9,rinse,Sample,457.683224,117.570355,130.702587,1758.471984,123.883572,0.000000,NaN,NaN,0.000000,0.000000,inf,69.192164,inf,inf,0.000000,32.664219,32.668027,0.000000,74.818069,9.018722,inf,23.182558,inf,inf,inf,12.315123,27.095413,0.000000,29.321230,49.204262,31.024046,57.179464


## 5. Save to Excel

In [32]:

# Define the Excel file path
excel_file_path = "results/EPA_1633_results.xlsx"
# Save DataFrames to separate sheets
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    # 2.1 RSE EIS compounds
    df_rse_eis.to_excel(writer, sheet_name="2.1_RSE_EIS_Compounds", index=False)
    # 2.2 RSE target analytes
    df_rse_target.to_excel(writer, sheet_name="2.2_RSE_target", index=False)
    # 3.1 Concentration EIS compounds
    eis_RFS.to_excel(writer, sheet_name="3.1.1_RF_Eis", index=False)
    calculated_cons_EIS.to_excel(writer, sheet_name="3.1.2_concentration_EIS", index=False)

    #3.2 Recoveries EIS compounds
    calculated_recoveries_EIS.to_excel(writer, sheet_name="3.2_recoveries_EIS", index=False)

    #4.1 Target analytes response factors 
    RF_target_analytes.to_excel(writer, sheet_name="4.1 Target_analytes_RF", index=False)
    # 4.2 Target analytes concentrations
    conc_target_analytes.to_excel(writer, sheet_name="4.2_Target_Conc", index=False)

print(f"DataFrames saved to {excel_file_path}")


DataFrames saved to results/EPA_1633_results.xlsx
